# **RETO HACKATON**: *CIENCIA DE DATOS - CAIXA BANK*
*Carlos Cabruja - Data*

## Background

El IBEX 35 es el índice oficial de la bolsa española compuesto por las 35 empresas más negociadas del mercado. Este índice nos muestra en tiempo real si los precios en bolsa están subiendo o bajando, por lo que permite medir el comportamiento de este conjunto de acciones.

El IBEX35 sirve como punto de referencia para los inversores del mercado español. La rentabilidad de este índice es el objetivo a batir por los gestores.

Por lo tanto, la modelización de las dinámicas de este tipo de índices resultan esenciales para la toma de decisiones por parte de todas las entidades bursátiles.

## Reto

1. Desarrolla un modelo predictivo que permita predecir la variable target (si el precio de cierre del IBEX35 será superior o inferior al precio de cierre actual).

Para ello deberas entrenar tu modelo con los datos de training (si también se usan los tweets se sumaran 100 puntos) e introducir como input de tu modelo el dataset test_x para realizar las predicciones.

2. Crea un breve documento (máx. 2 páginas) o presentación (máx. 4 slides) explicando la solución que has empleado y porque la has empleado.

## Librerías

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import nltk
import datetime as dt

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
import multiprocessing

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

## Recopilación y análisis de datos

In [ ]:
# cargamos los datasets a trabajar
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test_x.csv')
tweets = pd.read_csv('data/tweets_from2015_#Ibex35.csv')

In [ ]:
train # visualizamos los datos de entrenamiento

In [ ]:
test # visualizamos los datos de prueba

In [ ]:
tweets # visualizamos los datos de tweets

Estan correctamente cargados csv, ahora vamos su tipo y si presentan valores nulos.

In [ ]:
train.info()

Habrá que hacer tratamiento de nulls en los datos y convertir la columna **Date** en datetime

In [ ]:
test.info()

Parece ya encontrase limpia y lista para ser procesada.

In [ ]:
tweets.info()

Hacer **tweetdate** un Date para que pueda hacer merge con nuestro dataframe de train, y tratamiento de nulls

### Limpieza de datos

In [ ]:
# cuantos nulls tiene tweets?
tweets.isnull().sum()

In [ ]:
# filas con nulls en tweetDate
tweets[tweets.tweetDate.isnull()]

No hay nada, eliminamos....

In [ ]:
# eliminar filas con nulls en tweetDate
tweets = tweets.dropna(subset=['tweetDate'])

# filas con nulls en text
tweets[tweets.text.isnull()]

Tampoco son relevantes los tweets que no tienen fecha de publicación, ni texto de los tweets.

In [ ]:
# eliminar todos los nulls
tweets = tweets.dropna()
tweets.isnull().sum()

In [ ]:
# convertir Date a datetime YYYY-MM-DD
try:
    tweets['tweetDate'] = pd.to_datetime(tweets['tweetDate'])
except Exception as e:
    print(e)

Como **tweetdate** está contaminada con texto la vamos a tratar fila por fila. 

In [ ]:
for i in tweets['tweetDate']:
    lista_i = str(i).split(' ')
    if len(lista_i) != 6: # si no tiene 6 elementos (YYYY-MM-DD HH:MM:SS)
        # eliminar filas con fechas mal formateadas
        tweets = tweets.drop(tweets[tweets['tweetDate'] == i].index)

In [ ]:
# convertir Date a datetime YYYY-MM-DD
tweets['tweetDate'] = pd.to_datetime(tweets['tweetDate'])

In [ ]:
# renombrar tweetDate a Date
tweets = tweets.rename(columns={'tweetDate': 'Date'})

Y la columna handle no nos sirve ya que no daremos importancia a quién ha escrito el tweet.

In [ ]:
# eliminar handle de los tweets
tweets = tweets.drop(['handle'], axis=1)
tweets

Ahora creamos la columna sentimiento, con la libreria de Sentiment Analysis de NLTK.

**Análisis de sentimiento VADER:**
VADER (Valence Aware Dictionary and sEntiment Reasoner) es una herramienta de análisis de sentimientos basada en reglas y léxico que está específicamente en sintonía con los sentimientos expresados ​​en las redes sociales. VADER usa una combinación de un léxico de sentimientos es una lista de características léxicas (por ejemplo, palabras) que generalmente se etiquetan según su orientación semántica como positivas o negativas. VADER no solo informa sobre la puntuación de positividad y negatividad, sino que también nos informa sobre qué tan positivo o negativo es un sentimiento.

In [ ]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiment = SentimentIntensityAnalyzer() # instanciamos el analizador

In [ ]:
tweets['sentiment'] = tweets['text'].apply(lambda x: sentiment.polarity_scores(x)['compound'])
tweets

In [ ]:
# eliminar text
tweets = tweets.drop(['text'], axis=1)

Ahora, a fin de hacer el merge con train, trataremos los datos para que sea solo una fecha.

Para los tweets hechos en una misma fecha pero distinta hora, se hará una media de los sentimientos.

In [ ]:
# convertir Date a datetime YYYY-MM-DD
tweets['Date'] = pd.to_datetime(tweets['Date']).dt.date

In [ ]:
# eliminar Dates repetidos con el mismo sentimiento y fecha
tweets = tweets.drop_duplicates()

# hay alguna fecha repetida con distinto sentimiento?
tweets.groupby(['Date', 'sentiment']).count()

In [ ]:
# sacamos las fechas repetidas con distinto sentimiento
temp_df = tweets.groupby(['Date']).count()
lista_fechas = temp_df[temp_df['sentiment'] > 1].index

# restamos el indice de sentimiento en las fechas repetidas
for i in lista_fechas:
    # sacamos las filas repetidas
    temp_df = tweets[tweets['Date'] == i]
    # restamos el indice de sentimiento en las fechas repetidas
    sentiment = temp_df['sentiment'].mean()
    # actualizamos el sentimiento en las filas repetidas
    tweets.loc[tweets['Date'] == i, 'sentiment'] = sentiment

In [ ]:
# eliminamos las filas repetidas
tweets = tweets.drop_duplicates()

Ya tenemos nuestra tabla tweets limpia, así que hacemos el merge con train.

In [ ]:
# hacer train Date a datetime
train['Date'] = pd.to_datetime(train['Date']).dt.date
train

In [ ]:
# merge de train y tweets
train_tweets = pd.merge(train, tweets, on='Date')
train_tweets

Ahora vamos a limpiar nuestros datos de entrenamiento 

In [ ]:
train_tweets.isnull().sum()

In [ ]:
# muestra los nulos en open
train_tweets[train_tweets.Open.isnull()]

No contiene ninguna información relevante, por lo que la vamos a eliminar.

In [ ]:
train_tweets = train_tweets.dropna()
train_tweets.isnull().sum()

Como último paso vamos a categorizar la influencia de los tweets en neutral si el sentimiento es 0, positivo si es mayor a 0 y negativo si es menor a 0.

Esto lo harémos porque ya la influencia del tweet está documentada como importante que son más de 2 likes y 2 retweets, entonces aunque VADER nos dé un valor muy cercano al 0 (neutral) lo que queremos es determinar si la influencia de los tweets aporta valor a nuestro modelo, así que valoraremos esa diferencia del neutral. 

In [ ]:
# convertir sentiment a neutral, positivo y negativo
train_tweets['sentiment'] = train_tweets['sentiment'].apply(lambda x: 'neutral' if x == 0 else ('positivo' if x > 0 else 'negativo'))
train_tweets

In [ ]:
train_tweets.sentiment.value_counts()

Que la mayoría sea neutral, nos aportará una mayor información para determinar si esos tweets que valoramos como negativos o positivos indiferentemente de su distancia a la neutralidad tienen alguna influencia en el modelo y estudiar un modelo que tenga en cuenta cuanto porcentaje de negatividad o positividad tiene influencia en el indice IBEX35

In [ ]:
# guardamos los datos en un pickle
train_tweets.to_pickle('data/train_tweets.pkl')

Ahora hacemos el merge de los tweets con test

In [ ]:
# hacer test Date a datetime
test['Date'] = pd.to_datetime(test['Date']).dt.date
test

In [ ]:
# merge de test y tweets
test_tweets = pd.merge(test, tweets, on='Date')
test_tweets

In [ ]:
# convertir sentiment a neutral, positivo y negativo
test_tweets['sentiment'] = test_tweets['sentiment'].apply(lambda x: 'neutral' if x == 0 else ('positivo' if x > 0 else 'negativo'))
test_tweets

In [ ]:
# guardamos los datos en un pickle
test_tweets.to_pickle('data/test_tweets.pkl')

## Preprocesamiento

In [ ]:
# convertir sentiment a dummies
train_tweets = pd.get_dummies(train_tweets, columns=['sentiment'])
train_tweets

In [ ]:
test_tweets = pd.get_dummies(test_tweets, columns=['sentiment'])
test_tweets

In [ ]:
# estandarizacón de variables númericas
num_cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

# aplicar standarScaler a las variables númericas
scaler = StandardScaler()
train_tweets[num_cols] = scaler.fit_transform(train_tweets[num_cols])

train_tweets

In [ ]:
test_tweets[num_cols] = scaler.transform(test_tweets[num_cols])
prep_test = test_tweets.copy()
prep_test = prep_test.drop(['test_index'], axis=1)
prep_test = prep_test.set_index('Date')

In [ ]:
X = train_tweets.drop(['Target'], axis=1).set_index('Date')
y = train_tweets['Target']

## Modelización

### Random Forest Classifier

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators': [150],
              'max_depth'   : [None, 3, 10, 20],
              'criterion'   : ['gini', 'entropy']
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RandomForestClassifier(random_state = 123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=5, n_repeats=3, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X, y = y)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

In [ ]:
random_forest = grid.best_estimator_

### Gradient Boosting Classifier

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'n_estimators'  : [50, 100, 500, 1000],
              'max_features'  : ['auto', 'sqrt', 'log2'],
              'max_depth'     : [None, 1, 3, 5, 10, 20],
              'subsample'     : [0.5, 1],
              'learning_rate' : [0.001, 0.01, 0.1]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = GradientBoostingClassifier(random_state=123),
        param_grid = param_grid,
        scoring    = 'accuracy',
        n_jobs     = multiprocessing.cpu_count() - 1,
        cv         = RepeatedKFold(n_splits=3, n_repeats=1, random_state=123), 
        refit      = True,
        verbose    = 0,
        return_train_score = True
       )

grid.fit(X = X, y = y)

# Resultados
# ==============================================================================
resultados = pd.DataFrame(grid.cv_results_)
resultados.filter(regex = '(param*|mean_t|std_t)') \
    .drop(columns = 'params') \
    .sort_values('mean_test_score', ascending = False) \
    .head()

In [ ]:
# Mejores hiperparámetros por validación cruzada
# ==============================================================================
print("----------------------------------------")
print("Mejores hiperparámetros encontrados (cv)")
print("----------------------------------------")
print(grid.best_params_, ":", grid.best_score_, grid.scoring)

In [ ]:
gradient_boosting = grid.best_estimator_